In [1]:
import pandas as pd

df=pd.read_csv("./Data/Low2.csv")
df
# 열 Q, K1, K2, K3, K4의 내용을 합쳐서 하나의 열로 합치기
df['QK'] = df['Q'] +"? : 1."+ df['K1'] +", 2."+  df['K2'] + ", 3."+ df['K3'] +", 4."+  df['K4']
df=df[['QK','A']]
#열 이름을 text, prediction으로 고치기
df.columns=['text','prediction']
df


,text,prediction
0,"写真帳」を英語でいうと何 ?? : 1.loose leaf, 2.account book...",album
1,"鮭の内臓をとって、塩漬けにしたものを何という ?? : 1.さらまき鮭, 2.はらまき鮭, ...",あらまき鮭
2,"次のうち、約21kmの距離で行われるマラソン競技はどれ ? _? : 1.フルマフソン, ...",ハーフマラフソン
3,"秘の暑い時に、風を起こすために用いるものはどれ ?2? : 1.うちきさ, 2.うちわ, 3...",うちわ
4,"関門海峡を挟んで九州と面している山口県の都市はどこ ?? : 1.上関市, 2.下関市, 3...",下関市
...,...,...
22555,石田スイの漫画『東京喰種トーキョーグール』の舞台となっている国はとどこ?? : 1.オースト...,日本
22556,フジテレビのクイズ番組『ネプリーグ』」のボーナスステージといえば「何アドベンチャー」 2? ...,トロッコ
22557,"エアラインといえば、何の会証 ?? : 1.鉄道会計, 2.タクシー会視, 3.航空会計,...",航空会計
22558,"長崎に鳴滝塾を開いたシーボルトは円の ーー? : 1.中国人, 2.ドイツ人...",ドイツ人


In [2]:
import numpy as np
from datasets import load_dataset

train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.99 * len(df)), int(0.999 * len(df))]
)

print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

Source News : 次のうち、空気中で最も速く移動すとで/ぁ】ち円肝/? : 1.イチローのバックホーム, 2.リニアモーターカー, 3.光, 4.と=コ
四
Summarization : 光
Training Data Size : 22334
Validation Data Size : 203
Testing Data Size : 23


In [3]:
import torch
from transformers import BartTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence


def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text=data.text.tolist(),
        padding="longest",
        truncation=True,
        return_tensors="pt"
    )
    labels = []
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    for target in data.prediction:
        labels.append(tokenizer.encode(target, return_tensors="pt").squeeze())
    labels = pad_sequence(labels, batch_first=True, padding_value=-100).to(device)
    return TensorDataset(input_ids, attention_mask, labels)



def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 50
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = BartTokenizer.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(train_dataset[0], device)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(tensor([    0, 37127, 11582,  5543, 48018, 48718, 46311,  5543, 47341, 36714,
        15375,  3070, 48570,  6800, 47643, 48190, 44636,  7471, 48885, 48173,
         4333, 49040, 36714,  6248,  2023, 48675, 15722, 48735, 48281, 48190,
           73, 46311, 10172, 49287, 46311,  5543, 48745, 27819, 36484,  9264,
           46, 29482,  4832,   112,     4, 48611, 49511, 49323, 48438, 48018,
        49441, 49574, 48909,  3726, 48943, 21402,     6,   132,     4, 48947,
        49464, 49017, 49768, 48438, 49343, 48438, 49063, 48438,     6,   155,
            4, 49500,     6,   204,     4, 48281,  5214, 49195, 50121, 50118,
        47516,  3726,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,

In [4]:
from torch import optim
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
).to(device)
model.load_state_dict(torch.load("./Model/Bart.pt"))
optimizer = optim.Adam(model.parameters(), lr=5e-4, eps=1e-8)

In [5]:
for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)

model
└ shared
└ encoder
│  └ embed_tokens
│  └ embed_positions
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
│  │  └ 4
│  │  └ 5
│  └ layernorm_embedding
└ decoder
│  └ embed_tokens
│  └ embed_positions
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
│  │  └ 4
│  │  └ 5
│  └ layernorm_embedding
lm_head


In [6]:
import numpy as np
import evaluate
from tqdm import tqdm
import torch
import gc  # gc 패키지 임포트

def calc_rouge(preds, labels):
    preds = preds.argmax(axis=-1)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge2 = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    return rouge2["rouge2"]

def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for input_ids, attention_mask, labels in tqdm(dataloader, desc="Training"):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.zero_grad()
        
        optimizer.step()

        # GPU 캐시 비우기
        gc.collect()
        torch.cuda.empty_cache()

    train_loss = train_loss / len(dataloader)
    return train_loss

def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        val_loss, val_rouge = 0.0, 0.0

        for input_ids, attention_mask, labels in tqdm(dataloader, desc="Evaluation"):
            outputs = model(
                input_ids=input_ids, attention_mask=attention_mask, labels=labels
            )
            logits = outputs.logits
            loss = outputs.loss

            logits = logits.detach().cpu().numpy()
            label_ids = labels.to("cpu").numpy()
            rouge = calc_rouge(logits, label_ids)
            
            val_loss += loss
            val_rouge += rouge

        val_loss = val_loss / len(dataloader)
        val_rouge = val_rouge / len(dataloader)
        return val_loss, val_rouge

rouge_score = evaluate.load("rouge", tokenizer=tokenizer)
best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss, val_accuracy = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Rouge {val_accuracy:.4f}")

    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model.state_dict(), "./Model/Bart.pt")
        print("Saved the model weights")


Training:   0%|          | 0/2792 [00:00<?, ?it/s]C:\Users\mathn\AppData\Roaming\Python\Python38\site-packages\transformers\models\bart\modeling_bart.py:587: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Evaluation: 100%|██████████| 26/26 [00:06<00:00,  4.30it/s]


Epoch 1: Train Loss: 0.8143 Val Loss: 0.6997 Val Rouge 0.0000
Saved the model weights


Evaluation: 100%|██████████| 26/26 [00:05<00:00,  4.35it/s]


Epoch 2: Train Loss: 0.8150 Val Loss: 0.6997 Val Rouge 0.0000


Evaluation: 100%|██████████| 26/26 [00:05<00:00,  4.34it/s]


...

Epoch 14: Train Loss: 0.8170 Val Loss: 0.6997 Val Rouge 0.0000


Evaluation: 100%|██████████| 26/26 [00:05<00:00,  4.34it/s]


Epoch 15: Train Loss: 0.8153 Val Loss: 0.6997 Val Rouge 0.0000


Training:  12%|█▏        | 331/2792 [01:37<12:07,  3.38it/s]


KeyboardInterrupt: 

In [ ]:
model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
).to(device)
model.load_state_dict(torch.load("./Model/Bart.pt"))

test_loss, test_rouge_score = evaluation(model, test_dataloader)
print(f"Test Loss : {test_loss:.4f}")
print(f"Test ROUGE-2 Score : {test_rouge_score:.4f}")

Evaluation: 100%|██████████| 2/2 [00:00<00:00,  2.50it/s]

Test Loss : 0.5255
Test ROUGE-2 Score : 0.0000


In [7]:
from transformers import pipeline


summarizer = pipeline(
    task="summarization",
    model=model,
    tokenizer=tokenizer,
    max_length=54,
    device="cpu"
)

for index in range(2000,2005):
    news_text = df.text.iloc[index]
    summarization = df.prediction.iloc[index]
    predicted_summarization = summarizer(news_text)[0]["summary_text"]
    print(f"정답 요약문 : {summarization}")
    print(f"모델 요약문 : {predicted_summarization}\n")

정답 요약문 : 赤毛のアン
모델 요약문 :  2.白毛のアン

정답 요약문 : もののけ姫
모델 요약문 :  2.在成紳合戦ぼんぼこ

정답 요약문 : チョウ
모델 요약문 : 2.コオロギ, 3.ホタル

정답 요약문 : 和仁
に コ
모델 요약문 :  1.和仁
に コ

정답 요약문 : 新世紀エヴァンゲリオン
모델 요약문 : 2.宇宙戦艦マト

